In [3]:
import pandas as pd
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

Note: you may need to restart the kernel to use updated packages.Collecting pandas
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-2.2.0-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)

